In [ ]:
# import statements
import requests
import re
import json
from bs4 import BeautifulSoup as BS


while year < 2025:
    # construct url
    url = "https://en.wikipedia.org/wiki/1906_Tour_de_France"

In [ ]:
# scrape wikipedia page
def scrape(url):
    # need header to scrape from wikipedia
    headers = {
        'User-Agent': 'SchoolProjectBot/1.0 (r1043412@student.thomasmore.be) Python-Requests/2.31.0'
    }
    page = requests.get(url, headers=headers)
    
    return BS(page.text, "html")

In [ ]:
# take information from BeautifulSoup and clean it
def extract_clean_data(soup):
    # extract the table from the page html
    table = soup.find('table', class_ = 'wikitable')

    # use list comprehension instead of for loop
    courses = [
        # take all td's and put them in a list 'cells', take the second and third elements from that list
        (lambda cells: (cells[1].get_text(strip=True), cells[2].get_text(strip=True)))(row.find_all("td"))
        # take all rows exept title and results row
        for row in table.select('tr')[1:-1]
    ]

    # clean data so we can later perform analisys on it
    courses = [
        (lambda course:
        (course[0][:re.search('to[A-Z]', course[0]).start()], # start of course
        course[0][re.search('to[A-Z]', course[0]).end()-1:], # end of course
        course[1][:re.search(r'\D', course[1]).start()]) # distance of course
        )(course)
        for course in courses
    ]

    # transfer list of cleaned data to a dictionary so it can be written to json
    data_dict = [
        {'key': i, 'start': course[0], 'end': course[1], 'distance': course[2]}
        # use enumerate for the counter
        for i, course in enumerate(courses, start=1)
    ]

    return data_dict


In [ ]:
# write data to json
def write_json(dict, year, course_type):
    # open the json file and update it with the python dictionary
    with open(f'data/{course_type}.json', 'r+') as file:
        # load the entire json file into memory
        file_json = json.load(file)

        # update the json with the python dictionary
        file_json[course_type].update({year: dict})

        # put cursor at beginning of file and overwrite it with the dictionary in memory
        file.seek(0)
        json.dump(file_json, file, indent=4)